In [ ]:
%cd /content
!git clone https://github.com/hkozachkov/goom.git
%cd /content/goom
!git pull
!git checkout main
%pip install --upgrade .
%cd /content
!pytest goom/tests


In [ ]:
from goom.lmme import benchmark_lmme_implementations


In [ ]:
import jax
import goom
import goom.lmme
from goom.lmme import benchmark_lmme_implementations

print(jax.devices())
main_key = jax.random.PRNGKey(0)
# compare_lmme_implementations(main_key)
N = 1024
shapes = [(N,N,N)]
benchmark_lmme_implementations(main_key, shapes=shapes)

In [ ]:
import jax
import jax.numpy as jnp
from functools import partial
import numpy as np
import jax.experimental.pallas as pl
import jax.experimental.pallas.mosaic_gpu as plgpu
import goom.operations as ops

# even 256 is too big for the amount of shared memory colab gives me on 
# an L4 instance
N = 128

# yeah, this approach won't work with the goom functions
# NotImplementedError: Unimplemented primitive in Pallas GPU lowering: custom_vjp_call. Please file an issue on https://github.com/jax-ml/jax/issues.
def matmul_kernel(x_ref, y_ref, z_ref, *, activation):
  z_ref[...] = activation(x_ref[...] @ y_ref[...])

def matmul(x: jax.Array, y: jax.Array, *, activation):
  return pl.pallas_call(
    partial(matmul_kernel, activation=activation),
    out_shape=jax.ShapeDtypeStruct((x.shape[0], y.shape[1]), x.dtype),
    grid=(2, 2),
    in_specs=[
        pl.BlockSpec((x.shape[0] // 2, x.shape[1]), lambda i, j: (i, 0)),
        pl.BlockSpec((y.shape[0], y.shape[1] // 2), lambda i, j: (0, j))
    ],
    out_specs=pl.BlockSpec(
        (x.shape[0] // 2, y.shape[1] // 2), lambda i, j: (i, j)
    ),
  )(x, y)
k1, k2 = jax.random.split(jax.random.key(0))
x = jax.random.normal(k1, (N, N))
y = jax.random.normal(k2, (N, N))
# z = matmul(x, y, activation=jax.nn.relu)
z = matmul(x, y, activation=ops.from_goom)
# np.testing.assert_allclose(z, jax.nn.relu(x @ y))
